## Functional Call实现

### 定义工具

In [40]:
import os 

def write_file(path: str, content: str):
    """
        向路径path中写入content内容
    """
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True) if os.path.dirname(path) else None
        with open(path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"文件{path}已成功写入"
    except Exception as e:
        return  f"写入文件时发生异常-{str(e)}"

write_file("test.txt", "hello")
            
    

'文件test.txt已成功写入'

## 创建tools数组
利用JSON格式提供工具信息，主要格式如下：
```json
{
    "type": "function",
    "function": {
        "name": "函数名",
        "description": "工具函数的描述，大模型根据描述判断是否进行工具调用",
        "parameters": {          // 函数入参的描述
            "type": "object",
            "properties": {
                "location": {   // 输入参数的名称
                    "type": "string",  //  输入参数的类型
                    "description": "入参的描述。"
                }
            },
            "required": ["location"]  // 必须输入的参数
        }
    }
}
```

In [41]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "write_file",
            "description": "当想要创建文件，并往文件中写入内容时需要调用该函数。",
            "parameters":{
                "type": "object",
                "properties":{
                    "path":{
                        "type": "string",
                        "description":"需要创建文件的路径。"
                    },
                    "content":{
                        "type": "string",
                        "description":"需要写入文件的内容。"
                    }
                },
                "required":["path","content"]
            }
        }
    }
]

tool_name = [tool["function"]["name"] for tool in tools]
print(f"创建了{len(tools)}个tool，分别是{tool_name}")

创建了1个tool，分别是['write_file']


### 创建messages数组
**需要包含`system message`以及`user message`**

In [42]:
messages = [
    {
        "role": "system",
        "content":"你是一个很有帮助的助手，当用户要求创建文件并向文件中写入内容时，需要调用write_file函数。"
    },
    {
        "role":"user",
        "content":"给我一个实现了A*算法的Python文件，命名为A_star_algor.py,文件里代码相关的注释请用中文实现"
    }
]

### 发起Function Call

In [43]:
from openai import OpenAI

In [44]:
api_key = "sk-d97bb707eaec4049ae9e2818b32f080d"

In [45]:
client = OpenAI(
    api_key = api_key,
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
)

def functioncal_call():
    completions = client.chat.completions.create(
        model = "qwen-plus",
        messages = messages,
        tools= tools
    )
    print("返回对象为:")
    print(completions.choices[0].message.model_dump_json())
    print("\n")
    return completions

completion = functioncal_call()

返回对象为:
{"content":"","refusal":null,"role":"assistant","annotations":null,"audio":null,"function_call":null,"tool_calls":[{"id":"call_7590d70ad71841b59bbb9a","function":{"arguments":"{\"path\": \"A_star_algor.py\", \"content\": \"\\\"\\\"\\\"\\nA*算法实现\\n用于在网格地图中寻找从起点到终点的最短路径。\\n使用曼哈顿距离作为启发式函数。\\n\\\"\\\"\\\"\\n\\nclass Node:\\n    \\\"\\\"\\\"\\n    表示网格中的一个节点\\n    \\\"\\\"\\\"\\n    def __init__(self, position, parent=None):\\n        self.position = position  # 节点的位置，格式为 (行, 列)\\n        self.parent = parent      # 父节点，用于重构路径\\n        self.g = 0                # 从起点到当前节点的实际代价\\n        self.h = 0                # 启发式估计从当前节点到终点的代价\\n        self.f = 0                # 总代价 f = g + h\\n\\n    def __eq__(self, other):\\n        return self.position == other.position\\n\\n\\ndef heuristic(current, goal):\\n    \\\"\\\"\\\"\\n    计算启发式函数（曼哈顿距离）\\n    :param current: 当前节点位置 (row, col)\\n    :param goal: 目标节点位置 (row, col)\\n    :return: 曼哈顿距离\\n    \\\"\\\"\\\"\\n    return abs(current[0] 

In [48]:
import json

# 获取函数名称和入参
function_name = completion.choices[0].message.tool_calls[0].function.name
functioin_arguments = completion.choices[0].message.tool_calls[0].function.arguments

arguments = json.loads(functioin_arguments)

# 创建函数映射表

function_map = {
    "write_file": write_file
}

# 获取函数实体

function = function_map[function_name]

# print(**arguments)
if arguments == {}:
    output = function()
else:
    output = function(**arguments)
print(output)

文件A_star_algor.py已成功写入


### 将工具函数得到的结果返回给大模型，用于生成自然语言的回答

In [49]:
messages.append(completion.choices[0].message)
print("已添加assistant message")

messages.append({
    'role': 'tool',
    'content': output,
    'tool_call_id': completion.choices[0].message.tool_calls[0].id
})
print("已添加tool message")

已添加assistant message
已添加tool message


In [50]:
messages

[{'role': 'system',
  'content': '你是一个很有帮助的助手，当用户要求创建文件并向文件中写入内容时，需要调用write_file函数。'},
 {'role': 'user',
  'content': '给我一个实现了A*算法的Python文件，命名为A_star_algor.py,文件里代码相关的注释请用中文实现'},
 ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_7590d70ad71841b59bbb9a', function=Function(arguments='{"path": "A_star_algor.py", "content": "\\"\\"\\"\\nA*算法实现\\n用于在网格地图中寻找从起点到终点的最短路径。\\n使用曼哈顿距离作为启发式函数。\\n\\"\\"\\"\\n\\nclass Node:\\n    \\"\\"\\"\\n    表示网格中的一个节点\\n    \\"\\"\\"\\n    def __init__(self, position, parent=None):\\n        self.position = position  # 节点的位置，格式为 (行, 列)\\n        self.parent = parent      # 父节点，用于重构路径\\n        self.g = 0                # 从起点到当前节点的实际代价\\n        self.h = 0                # 启发式估计从当前节点到终点的代价\\n        self.f = 0                # 总代价 f = g + h\\n\\n    def __eq__(self, other):\\n        return self.position == other.position\\n\\n\\ndef heur

In [51]:
print("总结工具输出")
completion = functioncal_call()
print(completion.choices[0].message.content)

总结工具输出
返回对象为:
{"content":"文件 `A_star_algor.py` 已成功创建，并实现了 A* 算法。该文件包含详细的中文注释，便于理解代码逻辑。你可以直接运行此 Python 脚本进行测试，示例地图和起点终点已内置。\n\n### 功能说明：\n- 使用 **曼哈顿距离** 作为启发式函数。\n- 支持在二维网格中寻找从起点到终点的最短路径。\n- 障碍物用 `1` 表示，可通过区域用 `0` 表示。\n- 成功找到路径时返回坐标列表；失败则返回 `None`。\n\n你可以在其他项目中导入使用：\n```python\nfrom A_star_algor import a_star\n\ngrid = [[0, 0], [1, 0]]\npath = a_star(grid, (0, 0), (1, 1))\nprint(path)\n```\n\n如有需要扩展对角线移动、不同启发式函数等功能，也可以在此基础上修改。","refusal":null,"role":"assistant","annotations":null,"audio":null,"function_call":null,"tool_calls":null}


文件 `A_star_algor.py` 已成功创建，并实现了 A* 算法。该文件包含详细的中文注释，便于理解代码逻辑。你可以直接运行此 Python 脚本进行测试，示例地图和起点终点已内置。

### 功能说明：
- 使用 **曼哈顿距离** 作为启发式函数。
- 支持在二维网格中寻找从起点到终点的最短路径。
- 障碍物用 `1` 表示，可通过区域用 `0` 表示。
- 成功找到路径时返回坐标列表；失败则返回 `None`。

你可以在其他项目中导入使用：
```python
from A_star_algor import a_star

grid = [[0, 0], [1, 0]]
path = a_star(grid, (0, 0), (1, 1))
print(path)
```

如有需要扩展对角线移动、不同启发式函数等功能，也可以在此基础上修改。
